# 하이브리드 - Hybrid Search


# Hyrbid 검색이란?

하이브리드 검색은 키워드 검색과 Neural Search 검색을 결합하여 검색 관련성을 향상시킵니다. 하이브리드 검색을 구현하려면 검색 시간에 실행되는 [검색 파이프라인](https://opensearch.org/docs/latest/search-plugins/search-pipelines/index/)을 설정해야 합니다. 구성할 검색 파이프라인은 중간 단계에서 검색 결과를 가로채고 [`normalization_processor`](https://opensearch.org/docs/latest/search-plugins/search-pipelines/normalization-processor/)를 적용합니다. `normalization_processor`는 여러 쿼리 절에서의 문서 점수를 정규화하고 결합하여, 선택한 정규화 및 결합 기법에 따라 문서를 재점수화합니다.


# 사전 준비

이번 단계를 진행하기 위해서는 [시맨틱 검색 단계](./02.semantic_search.ipynb)를 필수적으로 완료하셔야 합니다. Amazon OpenSearch Service로의 연결은 [시맨틱 검색 단계](./02.semantic_search.ipynb)와 동일하게 수행합니다. 단 여기서는 코드를 간결하게 하기 위해 utils.py에 정의한 keyword_search와 semantic_search 함수를 재사용합니다.


In [1]:
# model_id 설정
%store -r model_id
%store -r index_name

In [2]:
print(model_id)
print(index_name)

o6EhfpEBhL3bNK7smYBX
movie_semantic


필요한 패키지를 설치합니다


### OpenSearch 도메인에 연결


OpenSearch 도메인에 접속하기 위한 호스트 정보와 인증정보를 가져옵니다


In [3]:
from opensearchpy import OpenSearch

# OpenSearch 연결 설정
base_url = "https://localhost:9200/_plugins/_ml"
host = 'localhost'
port = 9200
auth = ('admin', 'TestUser2@')  # 초기 설정한 어드민 비밀번호 사용

aos_client = OpenSearch(
    hosts=[{'host': host, 'port': port}],
    http_auth=auth,
    use_ssl=True,
    verify_certs=False,
    ssl_show_warn=False,
)

모델 아이디와 인덱스명을 설정하고 이 과정에서 사용할 인덱스가 잘 준비되어 있는지 확인합니다.


In [4]:
index_name = "movie_semantic"

count = aos_client.count(index=index_name)
print(count)

{'count': 1000, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}


# 검색 함수 정의하기


## 키워드 함수 정의


In [10]:
import pandas as pd

def keyword_search(query_text):
    query = {
        "size": 10,
        "_source": {"excludes": ["text", "vector_field"]},
        "query": {
            "multi_match": {
                "query": query_text,
                "fields": ["title", "plot", "genre", "main_act"],
            }
        },
    }

    res = aos_client.search(index=index_name, body=query)

    query_result = []
    for hit in res["hits"]["hits"]:
        row = [
            hit["_score"],
            hit["_source"]["title"],
            hit["_source"]["plot"],
            hit["_source"]["genre"],
            hit["_source"]["rating"],
            hit["_source"]["main_act"],
        ]
        query_result.append(row)

    query_result_df = pd.DataFrame(
        data=query_result, columns=["_score", "title", "plot", "genre", "rating", "main_act"]
    )
    display(query_result_df)

## 시맨틱 검색 함수 정의


In [11]:
def semantic_search(query_text):
    query = {
        "size": 10,
        "_source": {"excludes": ["text", "vector_field"]},
        "query": {
            "neural": {"vector_field": {"query_text": query_text, "model_id": model_id, "k": 10}},
        },
    }

    res = aos_client.search(index=index_name, body=query)

    query_result = []
    for hit in res["hits"]["hits"]:
        row = [
            hit["_score"],
            hit["_source"]["title"],
            hit["_source"]["plot"],
            hit["_source"]["genre"],
            hit["_source"]["rating"],
            hit["_source"]["main_act"],
        ]
        query_result.append(row)

    query_result_df = pd.DataFrame(
        data=query_result, columns=["_score", "title", "plot", "genre", "rating", "main_act"]
    )
    display(query_result_df)

## 하이브리드 검색 함수 정의

아래와 같이 Hybrid 검색 함수를 정의합니다. 중요한 부분은 다음과 같습니다.

1. **`query`** 부분에서 **`hybrid`** 쿼리를 사용하여 텍스트 기반 검색과 벡터 기반 검색을 결합합니다.
    - **`multi_match`** 쿼리는 **`title`**, **`text`**, **`genre`** 필드에서 **`query_text`**와 일치하는 문서를 찾습니다.
    - **`neural`** 쿼리는 **`vector_field`**에 저장된 벡터 데이터와 **`query_text`**를 사용하여 유사도 기반 검색을 수행합니다. **`model_id`**는 사용할 모델을 지정하고, **`k`**는 반환할 최대 문서 수를 지정합니다.
2. **`search_pipeline`**은 텍스트 기반 검색과 벡터 기반 검색의 결과를 결합하는 방식을 지정합니다.
    - **`normalization-processor`**는 두 검색 결과의 점수를 정규화합니다.
    - **`combination`** 부분에서 **`arithmetic_mean`** 기법을 사용하여 두 검색 결과의 점수를 가중 평균합니다. 여기서는 텍스트 기반 검색 결과에 0.3의 가중치를, 벡터 기반 검색 결과에 0.7의 가중치를 부여합니다.


In [12]:
def hybrid_search(query_text, keyword_weight=0.3, semantic_weight=0.7):
    query = {
        "size": 10,
        "_source": {"exclude": ["text", "vector_field"]},
        "query": {
            "hybrid": {
                "queries": [
                    {
                        "multi_match": {
                            "query": query_text,
                            "fields": ["title", "plot", "genre", "main_act", "supp_act"],
                        }
                    },
                    {
                        "neural": {
                            "vector_field": {
                                "query_text": query_text,
                                "model_id": model_id,
                                "k": 30,
                            }
                        }
                    },
                ]
            }
        },
        "search_pipeline": {
            "description": "Post processor for hybrid search",
            "phase_results_processors": [
                {
                    "normalization-processor": {
                        "normalization": {"technique": "min_max"},
                        "combination": {
                            "technique": "arithmetic_mean",
                            "parameters": {"weights": [keyword_weight, semantic_weight]},
                        },
                    }
                }
            ],
        },
    }

    res = aos_client.search(index=index_name, body=query)

    query_result = []
    for hit in res["hits"]["hits"]:
        row = [
            hit["_score"],
            hit["_source"]["title"],
            hit["_source"]["plot"],
            hit["_source"]["genre"],
            hit["_source"]["rating"],
            hit["_source"]["main_act"],
        ]
        query_result.append(row)

    query_result_df = pd.DataFrame(
        data=query_result, columns=["_score", "title", "plot", "genre", "rating", "main_act"]
    )
    display(query_result_df)

# 검색결과 비교하기

이전 단계에서 수행한 키워드 검색, 시맨틱 검색과 동일한 쿼리로 검색하여 결과를 비교해봅니다.


In [17]:
# query_text = "어벤져스와 비슷한 액션 SF 추천해줘"
query_text = "이병헌이 나오는 액션영화 추천해줘"

In [18]:
keyword_search(query_text)

,_score,title,plot,genre,rating,main_act
0,5.267291,늑대아이,"평범한 여대생 '하나'는 강의실에서 우연히 만나게 된 '그'에게 반하게 되고, 곧 ...",애니메이션|판타지|멜로/로맨스|드라마|모험|가족,9.12,미야자키 아오이|오오사와 타카오|쿠로키 하루|니시이 유키토|오노 모모카
1,4.909854,수상한 그녀,"아들 자랑이 유일한 낙인 욕쟁이 칠순 할매 오말순(나문희分)은 어느 날, 가족들이 ...",코미디|드라마,9.00,심은경|나문희|박인환|성동일|이진욱
2,4.748731,쏘우 2,심판이라는 이름으로 희생자를 속출시킨 희대의 지능적 살인마 직쏘! 잡힐 것 같지 않...,공포|스릴러,8.28,토빈 벨|쇼니 스미스|도니 월버그|에릭 넉슨|프랭키 G
3,4.323122,써니,전라도 벌교 전학생 나미는 긴장하면 터져 나오는 사투리 탓에 첫날부터 날라리들의 놀...,코미디|드라마,9.11,유호정|심은경|강소라|고수희|김민영|홍진희|박진주|이연경|남보라|김보미|민효린
4,3.761277,밀양,서른 세 살. 남편을 잃은 그녀는 아들 준과 남편의 고향인 밀양으로 가고 있다. 이...,드라마,7.89,전도연|송강호
5,3.253826,클래식,같은 대학에 다니는 지혜(손예진)와 수경은 연극반 선배 상민(조인성)을 좋아한다. ...,멜로/로맨스|드라마,9.39,손예진|조승우|조인성
6,3.253826,올드보이,술 좋아하고 떠들기 좋아하는 오.대.수. 본인의 이름풀이를 '오늘만 대충 수습하며 ...,드라마|미스터리|범죄|스릴러,9.04,최민식|유지태|강혜정


In [19]:
semantic_search(query_text)

,_score,title,plot,genre,rating,main_act
0,0.472819,"협녀, 칼의 기억","칼이 지배하던 시대, 고려 말 왕을 꿈꿨던 한 남자의 배신 그리고 18년 후 그를 ...",액션|드라마,5.36,이병헌|전도연|김고은
1,0.467313,비열한 거리,삼류조폭조직의 2인자 병두. 조직의 보스와 치고 올라오는 후배들 틈에서 제대로 된 ...,범죄|액션|느와르,8.79,조인성|천호진|남궁민|이보영
2,0.462453,내 사랑 내 곁에,몸이 조금씩 마비되어가는 루게릭병을 앓고 있는 종우(김명민). 유일한 혈육인 어머니...,드라마,7.03,하지원|김명민
3,0.453966,남한산성,1636년 인조 14년 병자호란. 청의 대군이 공격해오자 임금과 조정은 적을 피해 ...,드라마,8.08,이병헌|김윤석|박해일|고수|박희순
4,0.449317,오감도,‘segment 1 - His Concern’. 처음 만난 그와 그녀의 짜릿한 탐색...,멜로/로맨스,2.78,장혁|차현정|김강우|차수연|배종옥|김수로|김규리|엄정화|황정민|김효진|김동욱|신세경...
5,0.447815,기담,동경 유학 중이던 엘리트 의사 부부 인영(김보경)과 동원(김태우)은 갑작스레 귀국하...,공포,7.48,진구|이동규|김태우|김보경
6,0.447346,열정같은소리하고있네,취업만 하면 인생 제대로 즐기리라 생각한 햇병아리 연예부 수습기자 ‘도라희’(박보영...,드라마,7.39,정재영|박보영
7,0.447159,영화는 영화다,영화를 촬영하던 배우 장수타(강지환 扮)는 액션씬에서 욱하는 성질을 참지 못해 상대...,액션|범죄|드라마|느와르,8.85,소지섭|강지환
8,0.447143,독전,"의문의 폭발 사고 후, 오랫동안 마약 조직을 추적해온 형사 ‘원호’(조진웅)의 앞에...",범죄|액션,7.52,조진웅|류준열|김주혁|김성령|박해준
9,0.447037,달콤한 인생,"{어느 맑은 봄날, 바람에 이리저리 휘날리는 나뭇가지를 바라보며, 제자가 물었다. ...",느와르|액션|드라마,8.83,이병헌|김영철|신민아


In [20]:
hybrid_search(query_text, keyword_weight=0.5, semantic_weight=0.5)

,_score,title,plot,genre,rating,main_act
0,0.500000,"협녀, 칼의 기억","칼이 지배하던 시대, 고려 말 왕을 꿈꿨던 한 남자의 배신 그리고 18년 후 그를 ...",액션|드라마,5.36,이병헌|전도연|김고은
1,0.500000,늑대아이,"평범한 여대생 '하나'는 강의실에서 우연히 만나게 된 '그'에게 반하게 되고, 곧 ...",애니메이션|판타지|멜로/로맨스|드라마|모험|가족,9.12,미야자키 아오이|오오사와 타카오|쿠로키 하루|니시이 유키토|오노 모모카
2,0.419463,비열한 거리,삼류조폭조직의 2인자 병두. 조직의 보스와 치고 올라오는 후배들 틈에서 제대로 된 ...,범죄|액션|느와르,8.79,조인성|천호진|남궁민|이보영
3,0.411238,수상한 그녀,"아들 자랑이 유일한 낙인 욕쟁이 칠순 할매 오말순(나문희分)은 어느 날, 가족들이 ...",코미디|드라마,9.00,심은경|나문희|박인환|성동일|이진욱
4,0.371227,쏘우 2,심판이라는 이름으로 희생자를 속출시킨 희대의 지능적 살인마 직쏘! 잡힐 것 같지 않...,공포|스릴러,8.28,토빈 벨|쇼니 스미스|도니 월버그|에릭 넉슨|프랭키 G
5,0.348374,내 사랑 내 곁에,몸이 조금씩 마비되어가는 루게릭병을 앓고 있는 종우(김명민). 유일한 혈육인 어머니...,드라마,7.03,하지원|김명민
6,0.265536,써니,전라도 벌교 전학생 나미는 긴장하면 터져 나오는 사투리 탓에 첫날부터 날라리들의 놀...,코미디|드라마,9.11,유호정|심은경|강소라|고수희|김민영|홍진희|박진주|이연경|남보라|김보미|민효린
7,0.224238,남한산성,1636년 인조 14년 병자호란. 청의 대군이 공격해오자 임금과 조정은 적을 피해 ...,드라마,8.08,이병헌|김윤석|박해일|고수|박희순
8,0.156244,오감도,‘segment 1 - His Concern’. 처음 만난 그와 그녀의 짜릿한 탐색...,멜로/로맨스,2.78,장혁|차현정|김강우|차수연|배종옥|김수로|김규리|엄정화|황정민|김효진|김동욱|신세경...
9,0.134266,기담,동경 유학 중이던 엘리트 의사 부부 인영(김보경)과 동원(김태우)은 갑작스레 귀국하...,공포,7.48,진구|이동규|김태우|김보경


In [21]:
%store model_id

Stored 'model_id' (str)
